In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

# ---------- 各種関数の定義 ---------- #
# 可到達行列を導出する関数
def reachability_matrix(matrix):

    ###----------可達行列作成処理----------###
    # 行列のサイズを取得
    n = len(matrix)

    # print('\n----------隣接行列----------')
    matrix = matrix.astype(bool)
    I = np.eye(n).astype(bool)
    adj_matrix = np.logical_or(matrix, I)
    # print(adj_matrix.astype(int))

    # print('\n----------可達行列----------')
    reachability_matrix = adj_matrix.copy()
    while not np.array_equal(reachability_matrix, reachability_matrix @ adj_matrix):
        reachability_matrix = reachability_matrix @ adj_matrix  #@演算子を利用することで行列積を求める（bool演算）
    # print(reachability_matrix.astype(int))

    return reachability_matrix

# ノードの配置を取得する関数
def get_layer_positions(removed_rows_dict, custom_pos={}):
    positions = {}
    current_x = 0
    for layer, nodes in removed_rows_dict.items():
        for node in nodes:
            positions[node] = (current_x, -layer)
            current_x += 1
        current_x = 0

    # custom_posで指定したノード番号のx座標を辞書リストに追加する
    for node, x_coord in custom_pos.items():
        positions[node] = (x_coord, positions[node][1])

    return positions

# グラフオブジェクトを作成する関数
def create_graph(removed_rows_dict, matrix):    # matrixを追加
    G = nx.DiGraph()

    # ノードをグラフに追加
    for i in range(matrix.shape[0]):
        G.add_node(i)

    # matrixを参照しエッジを追加する関数
    def add_edges(source_node, target_nodes, target_layer_dict):
        sorted_target_nodes = sorted(target_nodes, key=lambda node: target_layer_dict[node], reverse=True)
        for target_node in sorted_target_nodes:
            if source_node != target_node and not nx.has_path(G, source_node, target_node) and not nx.has_path(G, target_node, source_node):
                G.add_edge(source_node, target_node)
                print('Edge added:', source_node, '->' , target_node)

    sorted_removed_rows_dict = dict(sorted(removed_rows_dict.items()))  # 階層を昇順にソート

    # target_nodeが属する階層の情報を辞書リストとして取得
    target_layer_dict = {}
    for layer, nodes in removed_rows_dict.items():
        for node in nodes:
            target_layer_dict[node] = layer
    
    print('\n--------エッジ追加処理--------')
    for layer, nodes in sorted_removed_rows_dict.items():
        target_nodes = []
        for node in nodes:
            target_nodes = matrix[node].nonzero()[0]
            add_edges(node, target_nodes, target_layer_dict)
    
    return G

# R, A列を取得する関数
def get_RorA(reachability_matrix):
    R_or_A = []
    for i in range(len(reachability_matrix)):
        temp = []
        for j in range(len(reachability_matrix[i])):
            if reachability_matrix[i][j] == 1:
                temp.append(j)
        R_or_A.append(temp)
    return R_or_A

# 重複する要素を抽出する関数
def extract_duplicates(list1, list2):
    # 各リスト内の各サブリストの集合を作成
    set_a = [set(sublist) for sublist in list1]
    set_b = [set(sublist) for sublist in list2]

    # 各リスト内で同じ位置にあるサブリストから重複する要素を抽出
    duplicates = [list(set_a[i] & set_b[i]) for i in range(min(len(set_a), len(set_b)))]

    return duplicates

# 特定の数字を除去する関数
def remove_number(lst, num):
    return [x for x in lst if x != num]

# 新しいデータフレームを取得する関数
def get_new_df(df, i, removed_rows_dict):
    remove_idx = []
    remove_id = []
    remove_num = []
    for j in range(len(df)):
        if df['R'][j] == df['RandA'][j]:
            remove_idx.append(j)
            remove_id.append(df['id'][j])
            for num in df['R'][j]:
                remove_num.append(num)

    remove_idx, remove_num
    # print('remove_idx:', remove_idx)
    # print('remove_num:', remove_num)
    removed_rows_dict[i] = remove_id

    # dfを更新
    df = df.drop(remove_idx)

    # 特定の数字をリスト内からすべて除去する
    for num in remove_num:
        to_remove = num
        # print(num)
        # df = df.applymap(lambda x: [elem for elem in x if elem != to_remove])
        # df = df.applymap(lambda x: remove_number(x, to_remove))
        df['R'] = df['R'].apply(lambda x: remove_number(x, to_remove))
        df['A'] = df['A'].apply(lambda x: remove_number(x, to_remove))
        df['RandA'] = df['RandA'].apply(lambda x: remove_number(x, to_remove))

    return df.reset_index(drop=True), removed_rows_dict


In [2]:
import os
# ---------- パスの取得 ---------- #
def get_path(folder_path):
    datapaths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            datapaths.append(os.path.join(root, file))

    return datapaths

folder_path = [r"C:\WorkSpace\collaborative_research\ISM_Analysis\ISMfiles\No.1", 
               r"C:\WorkSpace\collaborative_research\ISM_Analysis\ISMfiles\No.2"]

path = [get_path(folder_path[0]), 
         get_path(folder_path[1])]

In [3]:
# ---------- 入力データ確認 ---------- #
# 行列の読み取り条件
start_column = 2    # マトリックスの開始列（B列）
start_row = 2       # マトリックスの開始行（2行）
num = [29, 20, 88]            # 読み取るセルの範囲（num 行の正方形を読み取る）[A1, A2, B]の読み取る範囲

# 階層構造を定義した2次元マトリックスを取得する関数
def read_data(path, start_row, start_column, num):
    df = pd.read_excel(path, header=None, sheet_name="Sheet1", 
                       skiprows=start_row - 1, nrows=num, usecols=range(start_column - 1, start_column - 1 + num))
    
    data = np.where(df.values == 1, 1, 0)

    return data

# データを読み取る
matrix = [] # matrix[0]=No.1の行列リスト, matrix[1]=No.2の行列リスト,,,

for file in path:
    tmp = []    # No.n の行列を格納するリスト
    for i in range(len(file)):
        tmp.append(read_data(file[i], start_row, start_column, num[i]))
    matrix.append(tmp)

# matrix = read_data(path[0][0], start_row, start_column, num[0])
# print('---------- 入力データ ----------')
# print(matrix)

In [4]:
print(len(matrix), len(matrix[0]))

2 3


In [5]:
# ---------- 可達行列導出 ----------- #
reachability_matrix_list = []
for i in range(len(matrix)):
    tmp = []
    for j in range(len(matrix[i])):
        tmp_matrix = reachability_matrix(matrix[i][j])
        tmp_matrix = tmp_matrix.astype(int)
        tmp.append(tmp_matrix)
    reachability_matrix_list.append(tmp)

In [6]:
print(len(reachability_matrix_list), len(reachability_matrix_list[0]))

2 3


In [7]:
# ---------- データフレーム作成 ---------- #
df_list = []
for i in range(len(reachability_matrix_list)):
    tmp = []
    for j in range(len(reachability_matrix_list[i])):
        R = []
        A = []
        id = list(range(len(reachability_matrix_list[i][j])))

        R = get_RorA(reachability_matrix_list[i][j])
        A = get_RorA(np.transpose(reachability_matrix_list[i][j]))
        RandA = extract_duplicates(R, A)

        for RandA_i in RandA:
            RandA_i.sort()

        df = pd.DataFrame({'id':id, 'R':R, 'A':A, 'RandA':RandA})
        tmp.append(df)
    df_list.append(tmp)

In [8]:
len(df_list), len(df_list[0])

(2, 3)

In [9]:
# ---------- 関連度取得 ---------- #
d_add_r_list = []
for i in range(len(reachability_matrix_list)):
    tmp = []
    for j in range(len(reachability_matrix_list[i])):
        # 各行の和を取得
        row_sums = np.sum(reachability_matrix_list[i][j], axis=1)    # ISM
        # row_sums = np.sum(matrix, axis=1)   # dematel

        # 各列の和を取得
        column_sums = np.sum(reachability_matrix_list[i][j], axis=0) # ISM
        # column_sums = np.sum(matrix, axis=0)    # dematel

        # 関連度
        d_add_r = row_sums + column_sums
        tmp.append(d_add_r)
    d_add_r_list.append(tmp)

In [10]:
df_list[0][0]

,id,R,A,RandA
0,0,"[0, 6, 19, 23]","[0, 1, 4, 7, 10, 12, 17, 18, 19, 22, 24, 25, 2...","[0, 19]"
1,1,"[0, 1, 4, 6, 7, 10, 19, 23, 27, 28]","[1, 4, 7, 10, 12, 17, 18, 22, 27, 28]","[1, 4, 7, 10, 27, 28]"
2,2,[2],"[2, 5, 8, 12, 13, 14, 22]",[2]
3,3,[3],"[3, 17]",[3]
4,4,"[0, 1, 4, 6, 7, 10, 19, 23, 27, 28]","[1, 4, 7, 10, 12, 17, 18, 22, 27, 28]","[1, 4, 7, 10, 27, 28]"
5,5,"[2, 5, 8, 9, 13, 14, 15, 26]","[5, 8]","[5, 8]"
6,6,[6],"[0, 1, 4, 6, 7, 10, 12, 17, 18, 19, 22, 24, 25...",[6]
7,7,"[0, 1, 4, 6, 7, 10, 19, 23, 27, 28]","[1, 4, 7, 10, 12, 17, 18, 22, 27, 28]","[1, 4, 7, 10, 27, 28]"
8,8,"[2, 5, 8, 9, 13, 14, 15, 26]","[5, 8]","[5, 8]"
9,9,[9],"[5, 8, 9, 13, 15, 26]",[9]


In [11]:
# ---------- 階層を取得 ---------- #
levels_list = []
for i in range(len(df_list)):
    tmp = []
    for j in range(len(df_list[i])):
        levels = {}
        c = 0
        initial_df_length = len(df_list[i][j])

        for k in range(len(df_list[i][j])):
            df_list[i][j], levels = get_new_df(df_list[i][j], k+1, levels)
            # print('len(lebels): ', len(levels[i+1]))
            c += len(levels[k+1])
            # print('c: ', c)

            if c == initial_df_length:
                break
        tmp.append(levels)
    levels_list.append(tmp)

In [12]:
print(len(levels_list), len(levels_list[0]))

2 3


In [13]:
# ---------- ファイル名設定 ---------- #
def name(i):
    return f"fig\\No.{i+1}\\"
def name2(j):
    if j == 0:
        return "A1_"
    elif j == 1:
        return "A2_"
    else:
        return "B_"

In [14]:
d_add_r_list[0][0]

array([18, 20,  8,  3, 20, 10, 16, 20, 10,  7, 20,  4, 15,  8,  6,  7,  5,
       15, 15, 18,  4,  4, 14, 16,  6,  6,  7, 20, 20])

In [15]:
levels_list[0][0]

{1: [2, 3, 6, 9, 16, 20, 23],
 2: [0, 11, 14, 19, 26],
 3: [1, 4, 7, 10, 15, 21, 24, 25, 27, 28],
 4: [13, 18, 22],
 5: [5, 8, 12, 17]}

In [16]:
matrix[0][0].shape[0]

29

In [17]:
# ---------- ISMグラフ表示 ---------- #
for i in range(len(levels_list)):
    for j in range(len(levels_list[i])):
        # グラフを作成
        G = create_graph(levels_list[i][j], matrix[i][j])

        # ノードの配置を取得
        pos = get_layer_positions(levels_list[i][j])

        # グラフを描画して表示
        plt.figure(figsize=(16, 16), dpi=600)
        nx.draw(G, pos, with_labels=True, node_size=1500, node_color="skyblue", font_size=10, font_weight="bold", arrowsize=15)
        plt.savefig(name(i)+name2(j)+'ISM-MODEL.png', format='png')
        plt.show()

        # ---------- ノード位置の調整 ---------- #
        # custom_posでノード番号に対応するx座標を指定する
        # 自分で設定する場合
        custom_pos = {3 : 1, 
                    4 : 2,
                    11: -1,
                    16: 0}

        # 関与度からcustom_posを取得する場合
        custom_pos = {}
        for k in range(len(matrix[i][j])):
            custom_pos[k] = d_add_r_list[i][j][k]

        # ノードの配置を取得
        pos_fix = get_layer_positions(levels_list[i][j], custom_pos)

        # グラフを描画して表示
        plt.figure(figsize=(12, 10), dpi=600)
        nx.draw(G, pos_fix, with_labels=True, node_size=1500, node_color="skyblue", font_size=10, font_weight="bold", arrowsize=15)
        plt.savefig(name(i)+name2(j)+'ISM-MODEL_adjustment.png', format='png')
        plt.show()

# # グラフを作成
# G = create_graph(levels_list[0][0], matrix[0][0])

# # ノードの配置を取得
# pos = get_layer_positions(levels_list[0][0])

# # グラフを描画して表示
# plt.figure(figsize=(8, 8), dpi=600)
# nx.draw(G, pos, with_labels=True, node_size=1500, node_color="skyblue", font_size=10, font_weight="bold", arrowsize=15)
# plt.savefig('fig\ISM-MODEL_a-2v2.png', format='png')
# plt.show()

# # ---------- ノード位置の調整 ---------- #
# # custom_posでノード番号に対応するx座標を指定する
# # 自分で設定する場合
# # custom_pos = {3 : 1, 
# #               4 : 2,
# #               11: -1,
# #               16: 0}

# # 関与度からcustom_posを取得する場合
# custom_pos = {}
# for i in range(len(matrix)):
#     custom_pos[i] = d_add_r_list[0][0][i]

# # ノードの配置を取得
# pos_fix = get_layer_positions(levels, custom_pos)

# # グラフを描画して表示
# plt.figure(figsize=(12, 10), dpi=600)
# nx.draw(G, pos_fix, with_labels=True, node_size=1500, node_color="skyblue", font_size=10, font_weight="bold", arrowsize=15)
# plt.savefig('fig\ISM-MODEL_adjustment_a-2v2.png', format='png')
# plt.show()




# # グラフを作成
# G = create_graph(levels)

# # ノードの配置を取得
# pos = get_layer_positions(levels)

# # グラフを描画して表示
# plt.figure(figsize=(8, 8), dpi=600)
# nx.draw(G, pos, with_labels=True, node_size=1500, node_color="skyblue", font_size=10, font_weight="bold", arrowsize=15)
# plt.savefig('fig\ISM-MODEL_a-2v2.png', format='png')
# plt.show()

# # ---------- ノード位置の調整 ---------- #
# # custom_posでノード番号に対応するx座標を指定する
# # 自分で設定する場合
# custom_pos = {3 : 1, 
#               4 : 2,
#               11: -1,
#               16: 0}

# # 関与度からcustom_posを取得する場合
# custom_pos = {}
# for i in range(len(matrix)):
#     custom_pos[i] = d_add_r[i]

# # ノードの配置を取得
# pos_fix = get_layer_positions(levels, custom_pos)

# # グラフを描画して表示
# plt.figure(figsize=(12, 10), dpi=600)
# nx.draw(G, pos_fix, with_labels=True, node_size=1500, node_color="skyblue", font_size=10, font_weight="bold", arrowsize=15)
# plt.savefig('fig\ISM-MODEL_adjustment_a-2v2.png', format='png')
# plt.show()


--------エッジ追加処理--------
Edge added: 0 -> 19
Edge added: 0 -> 6
Edge added: 11 -> 20
Edge added: 14 -> 2
Edge added: 19 -> 23
Edge added: 26 -> 9
Edge added: 1 -> 10
Edge added: 1 -> 27
Edge added: 1 -> 0
Edge added: 4 -> 7
Edge added: 4 -> 27
Edge added: 4 -> 28
Edge added: 7 -> 28
Edge added: 10 -> 0
Edge added: 15 -> 26
Edge added: 21 -> 11
Edge added: 24 -> 0
Edge added: 25 -> 0
Edge added: 27 -> 7
Edge added: 27 -> 0
Edge added: 28 -> 10
Edge added: 13 -> 15
Edge added: 13 -> 2
Edge added: 18 -> 27
Edge added: 18 -> 16
Edge added: 22 -> 28
Edge added: 22 -> 14
Edge added: 5 -> 8
Edge added: 5 -> 13
Edge added: 5 -> 14
Edge added: 12 -> 18
Edge added: 12 -> 2
Edge added: 17 -> 18
Edge added: 17 -> 3


FileNotFoundError: [Errno 2] No such file or directory: 'fig\\No.1\\A1_ISM-MODEL.png'